# Purpose
- This file is a scratch-pad for sql queries

In [8]:
%load_ext sql
%sql duckdb:///../data/referral.db --alias referralsDB

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [9]:
%sql referralsDB

In [16]:
%%sql 
CREATE OR REPLACE TABLE team as 

SELECT DISTINCT * FROM read_csv_auto('../data/reference/referral_team.csv', header=True)

*  (referralsDB) duckdb:///../data/referral.db
Done.


Count
16


In [11]:
%sql CREATE OR REPLACE TABLE hcpcs as SELECT DISTINCT * FROM read_csv_auto('../data/reference/cpt_hcpcs.csv', header=True)

*  (referralsDB) duckdb:///../data/referral.db
Done.


Count
16991


In [12]:
%sql CREATE OR REPLACE TABLE icd10cm as SELECT DISTINCT * FROM read_csv('../data/reference/icd10cm.csv', header=True, columns={'code_value': 'VARCHAR', 'code_long_description': 'VARCHAR'})

*  (referralsDB) duckdb:///../data/referral.db
Done.


Count
95387


In [21]:
%%sql 
CREATE OR REPLACE TABLE cln_referrals as 

SELECT DISTINCT * FROM read_parquet('../data/clean_referrals/*.parquet')

*  (referralsDB) duckdb:///../data/referral.db
Done.


Count
77610


- No Auth Required ->	Appointment Scheduled or Member to Schedule or Unable to Contact
- Approved	       ->   Appointment Scheduled or Member to Schedule or Unable to Contact
- Denied	       ->   Open
- Withdrawn	       ->   Open

In [23]:
%%sql 
CREATE OR REPLACE TABLE bi_referrals as

    SELECT DISTINCT *
    , COALESCE(tm.Team, 'N/A') as User_Team

    , UPPER("Last Name") || UPPER("First Name") as patient_name

    , UPPER("Last Name") || UPPER("First Name") || "DOB" as patient_id

    , patient_id || coalesce("Plan ID", 'NA') || coalesce("Referring Provider NPI", 'NA') || "Referral Date" || coalesce("Procedure", 'NA') || coalesce(Specialty, 'NA') as Referral_keyid

    , CASE
        WHEN "Approval Status" IN ('No HP Auth Required', 'HP Approved', 'Approved (comments required)', 'Complete/no Auth# needed')  
            AND "Visit Status" IN ('Appointment Scheduled', 'Member to Schedule', 'Unable to Contact', 'Denied', 'Withdrawn')
            THEN 'Completed'
        WHEN "Approval Status" IN ('Denied- Per Insurance Plan', 'Denied- Per Medical Director Review')
            AND "Visit Status" = 'Open'
            THEN 'Completed'
        ELSE "Referral Status" END as "Updated Status"

    , RIGHT("Procedure", 5) as Proc_code
    , pc.code_short_description as Proc_name 

    , FORMAT(
        '({}) {}-{}'
        , substring("Mobile Phone", 1, 3), substring("Mobile Phone", 3, 3), substring("Mobile Phone", 6, 4)
    ) as "Fmt Mobile Phone"

    , FORMAT(
        '({}) {}-{}'
        , substring("Home Phone", 1, 3), substring("Home Phone", 3, 3), substring("Home Phone", 6, 4)
    ) as "Fmt Home Phone"

    , icd.code_long_description AS "Diagnosis Description"

    FROM cln_referrals AS rf

    LEFT JOIN icd10cm AS icd
        ON trim(rf."Diagnosis") = trim(icd.code_value)

    LEFT JOIN hcpcs AS pc
        on RIGHT("Procedure", 5) = pc.code_value

    LEFT JOIN team as tm
        on  "User_FName" = tm.FName
        AND "User_LName" = tm.LName

    WHERE "Visit Status" IS NOT NULL

*  (referralsDB) duckdb:///../data/referral.db


FloatProgress(value=0.0, layout=Layout(width='100%'), style=ProgressStyle(bar_color='black'))

Done.


Count
77610


In [ ]:
%%sql 
COPY (SELECT * FROM bi_referrals) TO '../data/report_data/2023-04-06.parquet' (FORMAT 'parquet'); 

In [38]:
%%sql
COPY (SELECT DISTINCT "User_FName", "User_LName" FROM bi_referrals) to '../data/reference/user_list.csv' (FORMAT 'csv')

*  (referralsDB) duckdb:///../data/referral.db
Done.


Count
267
